# Week 3

Context information: The dataset I used is the Statlog Heart Disease dataset from Kaggle, which includes 14 features related to cardiovascular health. The chosen project was based on cardiovascular disease (CVD). This project examines data related to CVD in order to analyze healthcare utilization and risk factors. The dataset, Statlog Heart Disease, included medical attributes from 270 individuals. These included blood pressure, cholesterol, and heart rate, utilizing said data to predict the presence of heart disease. The project strived to discover trends in healthcare to identify key risk factors for cardiovascular disease and thus inform on new strategies for disease prevention and management.

In [1]:
!pip install statsmodels==0.14.4


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm

In [3]:
df = pd.read_csv('Heart_disease_statlog.csv')

In [4]:
print("Shape:", df.shape)
display(df.head())
display(df.describe())
print("\nMissing values:\n", df.isnull().sum())
print("\nColumn names:\n", df.columns)

Shape: (270, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,70,1,3,130,322,0,2,109,0,2.4,1,3,1,1
1,67,0,2,115,564,0,2,160,0,1.6,1,0,3,0
2,57,1,1,124,261,0,0,141,0,0.3,0,0,3,1
3,64,1,3,128,263,0,0,105,1,0.2,1,1,3,0
4,74,0,1,120,269,0,2,121,1,0.2,0,1,1,0


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.00000,270.000000,270.000000,270.000000,270.000000
mean,54.433333,0.677778,2.174074,131.344444,249.659259,0.148148,1.022222,149.677778,0.329630,1.05000,0.585185,0.670370,1.822222,0.444444
std,9.109067,0.468195,0.950090,17.861608,51.686237,0.355906,0.997891,23.165717,0.470952,1.14521,0.614390,0.943896,0.959140,0.497827
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000
25%,48.000000,0.000000,2.000000,120.000000,213.000000,0.000000,0.000000,133.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000
50%,55.000000,1.000000,2.000000,130.000000,245.000000,0.000000,2.000000,153.500000,0.000000,0.80000,1.000000,0.000000,1.000000,0.000000
75%,61.000000,1.000000,3.000000,140.000000,280.000000,0.000000,2.000000,166.000000,1.000000,1.60000,1.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.20000,2.000000,3.000000,3.000000,1.000000



Missing values:
 age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

Column names:
 Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')


In [5]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_df = pd.DataFrame(X_train, columns=X.columns)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)

Forward Selection (using AIC)

In [8]:
def forward_selection(X, y):
    initial_features = []
    remaining = list(X.columns)
    best_features = []
    
    while remaining:
        aic_with_candidates = []
        for candidate in remaining:
            features = initial_features + [candidate]
            X_const = sm.add_constant(X[features])
            model = sm.OLS(y, X_const).fit()
            aic_with_candidates.append((model.aic, candidate))
        
        aic_with_candidates.sort()
        best_new_feature = aic_with_candidates[0][1]
        initial_features.append(best_new_feature)
        remaining.remove(best_new_feature)
        best_features.append((aic_with_candidates[0][0], list(initial_features)))
    
    return best_features[-1][1]

selected_forward = forward_selection(X_train_df, y_train)
print("Forward Selection Chose:", selected_forward)

X_train_fs = sm.add_constant(X_train_df[selected_forward])
X_test_fs = sm.add_constant(X_test[selected_forward])
model_fs = sm.OLS(y_train, X_train_fs).fit()
y_pred_fs = model_fs.predict(X_test_fs)

print("\nForward Selection Model Performance:")
print(f"R²: {r2_score(y_test, y_pred_fs):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_fs)):.4f}")

Forward Selection Chose: ['thal', 'oldpeak', 'cp', 'ca', 'exang', 'sex', 'trestbps', 'slope', 'fbs', 'chol', 'thalach', 'age', 'restecg']

Forward Selection Model Performance:
R²: 0.5463
RMSE: 0.3284


Backward Elimination (using AIC)

In [11]:
def backward_elimination(X, y):
    features = list(X.columns)
    while len(features) > 0:
        X_const = sm.add_constant(X[features])
        model = sm.OLS(y, X_const).fit()
        aic = model.aic
        pvalues = model.pvalues.iloc[1:]  # Skip constant
        
        if pvalues.max() > 0.05:
            worst_feature = pvalues.idxmax()
            features.remove(worst_feature)
        else:
            break
    return features

selected_backward = backward_elimination(X_train_df, y_train)
print("Backward Elimination Chose:", selected_backward)

X_train_bs = sm.add_constant(X_train_df[selected_backward])
X_test_bs = sm.add_constant(X_test[selected_backward])
model_bs = sm.OLS(y_train, X_train_bs).fit()
y_pred_bs = model_bs.predict(X_test_bs)

print("\nBackward Elimination Model Performance:")
print(f"R²: {r2_score(y_test, y_pred_bs):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_bs)):.4f}")

Backward Elimination Chose: ['sex', 'cp', 'exang', 'oldpeak', 'ca', 'thal']

Backward Elimination Model Performance:
R²: 0.5108
RMSE: 0.3410


Principal Component Regression (PCR)

In [14]:
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

mse_list = []
for i in range(1, X.shape[1] + 1):
    model = LinearRegression()
    score = -1 * cross_val_score(model, X_train_pca[:, :i], y_train, scoring='neg_mean_squared_error', cv=5).mean()
    mse_list.append(score)

optimal_components = np.argmin(mse_list) + 1
print(f"Optimal # of components (PCR): {optimal_components}")

model_pcr = LinearRegression()
model_pcr.fit(X_train_pca[:, :optimal_components], y_train)
y_pred_pcr = model_pcr.predict(X_test_pca[:, :optimal_components])

print("\nPCR Model Performance:")
print(f"R²: {r2_score(y_test, y_pred_pcr):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_pcr)):.4f}")

Optimal # of components (PCR): 5

PCR Model Performance:
R²: 0.5776
RMSE: 0.3168


Partial Least Squares Regression (PLSR)

In [17]:
mse_pls = []
for i in range(1, X.shape[1] + 1):
    pls = PLSRegression(n_components=i)
    score = -1 * cross_val_score(pls, X_train_scaled, y_train, scoring='neg_mean_squared_error', cv=5).mean()
    mse_pls.append(score)

optimal_pls = np.argmin(mse_pls) + 1
print(f"Optimal # of components (PLSR): {optimal_pls}")

pls_final = PLSRegression(n_components=optimal_pls)
pls_final.fit(X_train_scaled, y_train)
y_pred_pls = pls_final.predict(X_test_scaled)

print("\nPLSR Model Performance:")
print(f"R²: {r2_score(y_test, y_pred_pls):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_pls)):.4f}")

Optimal # of components (PLSR): 1

PLSR Model Performance:
R²: 0.5236
RMSE: 0.3365


Performance Comparison Summary

In [20]:
results = pd.DataFrame({
    'Model': ['Forward Selection', 'Backward Elimination', 'PCR', 'PLSR'],
    'R² Score': [
        r2_score(y_test, y_pred_fs),
        r2_score(y_test, y_pred_bs),
        r2_score(y_test, y_pred_pcr),
        r2_score(y_test, y_pred_pls)
    ],
    'RMSE': [
        np.sqrt(mean_squared_error(y_test, y_pred_fs)),
        np.sqrt(mean_squared_error(y_test, y_pred_bs)),
        np.sqrt(mean_squared_error(y_test, y_pred_pcr)),
        np.sqrt(mean_squared_error(y_test, y_pred_pls))
    ]
})

print("\nModel Comparison:")
display(results)


Model Comparison:


,Model,R² Score,RMSE
0,Forward Selection,0.546265,0.328378
1,Backward Elimination,0.510773,0.340979
2,PCR,0.577610,0.316832
3,PLSR,0.523588,0.336484
